# ts2net Quick Start

This notebook demonstrates basic usage of ts2net with synthetic data.

## What is ts2net?

ts2net converts time series into networks using various methods:
- **Visibility Graphs**: Horizontal (HVG) and Natural (NVG) visibility graphs
- **Recurrence Networks**: Phase space embedding with recurrence analysis  
- **Transition Networks**: Symbolic dynamics and state transitions

## Installation

```bash
pip install ts2net
```


In [ ]:
import numpy as np
from ts2net import HVG, build_network, graph_summary

# Generate synthetic time series (random walk with trend)
np.random.seed(42)
n = 1000
x = np.cumsum(np.random.randn(n)) + 0.01 * np.arange(n)  # Random walk with drift

print(f"Time series length: {len(x)}")
print(f"First 10 values: {x[:10]}")


## Basic HVG (Horizontal Visibility Graph)

The simplest way to build a graph from a time series.


In [ ]:
hvg = HVG()
hvg.build(x)

print(f"Nodes: {hvg.n_nodes}")
print(f"Edges: {hvg.n_edges}")
print(f"Average degree: {np.mean(hvg.degree_sequence()):.2f}")


## Adjacency Matrix

Get the adjacency matrix representation of the graph.


In [ ]:
A = hvg.adjacency_matrix()
density = hvg.n_edges / (hvg.n_nodes * (hvg.n_nodes - 1) / 2)

print(f"Adjacency matrix shape: {A.shape}")
print(f"Graph density: {density:.4f}")
print(f"Matrix is symmetric: {np.allclose(A, A.T)}")


## NetworkX Conversion

Convert to NetworkX for further analysis.


In [ ]:
G_nx = hvg.as_networkx()
print(f"NetworkX graph type: {type(G_nx).__name__}")
print(f"Nodes: {G_nx.number_of_nodes()}, Edges: {G_nx.number_of_edges()}")

# Get network summary
print("\nNetwork Summary:")
print(graph_summary(G_nx))


## Performance Mode

For large time series, use `output='degrees'` to only compute degrees (faster, less memory).


In [ ]:
hvg_fast = HVG(output="degrees")
hvg_fast.build(x)
degrees = hvg_fast.degree_sequence()

print(f"Computed {len(degrees)} degrees")
print(f"Edges stored: {hvg_fast.edges is not None}")
print(f"Average degree: {np.mean(degrees):.2f}")


## Compare Different Methods

ts2net supports multiple network construction methods. Let's compare them:


In [ ]:
methods = {
    'hvg': {},
    'nvg': {},
    'recurrence': {'m': 3, 'rule': 'knn', 'k': 5},
    'transition': {'symbolizer': 'ordinal', 'order': 3}
}

print("Comparing network construction methods:\n")
for method, kwargs in methods.items():
    g = build_network(x, method, **kwargs)
    print(f"{method:12s}: {g.n_edges:6d} edges, {g.n_nodes:5d} nodes")
